<a href="https://colab.research.google.com/github/nicolasrojasv/nicolasrojasv.github.io/blob/main/predicting_longevity_prize_money.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Predicing the Longevity of a Career in the ATP Tour using Price Money**

In [1]:
# Import necessary libraries for data handling and visualization
import pandas as pd
import numpy as np
import altair as alt
import requests

In [2]:
# Load ATP players activity data from a CSV file
players_activity = pd.read_csv('https://raw.githubusercontent.com/nicolasrojasv/nicolasrojasv.github.io/refs/heads/main/Project/data/players_activity.csv')

In [3]:
# Display the first few rows of the players activity DataFrame
display(players_activity.head())

,player_id,year,event_id,event_name,event_title,prize_raw,currency,prize_usd
0,sy30,2019,8332,M25 Claremont,M25 Claremont,260,$,260
1,sy30,2019,1592,M25 Harlingen,M25 Harlingen,730,$,730
2,sy30,2019,8315,M15 Cancun,M15 Cancun,258,$,258
3,sy30,2019,8308,M15 Cancun,M15 Cancun,258,$,258
4,sy30,2019,8306,M15 Cancun,M15 Cancun,753,$,753


In [4]:
# Fetch US CPI data from FRED API, clean, and prepare it for analysis
API_KEY = "0720020a9e966c086f32e8200cf16b38"

url = "https://api.stlouisfed.org/fred/series/observations"
params = {
    "series_id": "CPIAUCSL",
    "api_key": API_KEY,
    "file_type": "json",
    "observation_start": "1985-01-01",
    "observation_end": "2025-12-31"
}

response = requests.get(url, params=params)
data = response.json()["observations"]

cpi = pd.DataFrame(data)
cpi["date"] = pd.to_datetime(cpi["date"])
cpi["cpi"] = pd.to_numeric(cpi["value"], errors="coerce")

cpi = cpi.dropna(subset=["cpi"])
cpi["year"] = cpi["date"].dt.year

In [5]:
# Calculate annual average CPI and inflation deflator relative to 2025
cpi_annual = (
    cpi
    .groupby("year", as_index=False)
    .agg(cpi_avg=("cpi", "mean"))
)
cpi_2025 = cpi_annual.loc[cpi_annual["year"] == 2025, "cpi_avg"].iloc[0]

cpi_annual["deflator_2025"] = cpi_2025 / cpi_annual["cpi_avg"]

In [6]:
# Display the annual CPI data with the calculated deflator
cpi_annual

,year,cpi_avg,deflator_2025
0,1985,107.600000,2.988636
1,1986,109.691667,2.931647
2,1987,113.616667,2.830370
3,1988,118.275000,2.718894
4,1989,123.941667,2.594585
5,1990,130.658333,2.461207
6,1991,136.166667,2.361644
7,1992,140.308333,2.291932
8,1993,144.475000,2.225833
9,1994,148.225000,2.169521


In [7]:
# Filter out specific prize categories from the player activity data
exclude = ["500 Bonus Pool", "1000 Bonus Pool", "Profit Sharing", "Profit Share", "Bonus Prize Money"]
players_activity_filtered = players_activity[~players_activity["event_name"].isin(exclude)]

In [8]:
# Calculate total prize money per player per year and display the last 10 rows
players_prize_by_year = players_activity_filtered.groupby(["player_id", "year"])["prize_usd"].sum().reset_index()
players_prize_by_year.tail(10)

,player_id,year,prize_usd
42995,z450,2017,1470
42996,z450,2018,2274
42997,z450,2019,1666
42998,z452,2016,812
42999,z452,2017,1704
43000,z452,2018,1804
43001,z457,2015,182
43002,z457,2016,104
43003,z460,2014,414
43004,z460,2015,516


In [9]:
# Sort the player prize money data by year and prize amount
players_prize_by_year = players_prize_by_year.sort_values(by=['year', 'prize_usd'], ascending=False)
players_prize_by_year.tail(10)

,player_id,year,prize_usd
10458,e121,1991,5366
34718,s535,1991,1040
23876,m343,1991,520
10457,e121,1990,1820
23875,m343,1990,1300
28153,o111,1989,212
23874,m343,1988,1194
23873,m343,1987,540
23872,m343,1985,680
3732,bf71,1956,0


In [10]:
# Adjust player prize money for inflation to 2025 USD
player_prize_money_adjust = players_prize_by_year.merge(
    cpi_annual[["year", "deflator_2025"]],
    on="year",
    how="left"
)

player_prize_money_adjust["prize_usd_2025"] = player_prize_money_adjust["prize_usd"] * player_prize_money_adjust["deflator_2025"]

In [11]:
# Round adjusted prize money and display a sample for 2004
player_prize_money_adjust["prize_usd_2025"] = player_prize_money_adjust["prize_usd_2025"].round(0)
player_prize_money_adjust.loc[player_prize_money_adjust["year"] == 2004].head()

,player_id,year,prize_usd,deflator_2025,prize_usd_2025
41591,f324,2004,6337660,1.702292,10788550.0
41592,h432,2004,2762924,1.702292,4703304.0
41593,r419,2004,746909,1.702292,1271457.0
41594,y061,2004,681469,1.702292,1160059.0
41595,l397,2004,647936,1.702292,1102976.0


In [12]:
# Determine player status (active/retired) based on activity in 2025
active_players_2025 = set(
    player_prize_money_adjust
    .loc[player_prize_money_adjust["year"] == 2025, "player_id"]
)

player_prize_money_adjust["status"] = np.where(
    player_prize_money_adjust["player_id"].isin(active_players_2025),
    "active",
    "retired"
)

In [13]:
# Count the number of active and retired players
player_prize_money_adjust["status"].value_counts()

,count
status,
retired,26639
active,16366


In [14]:
# Count unique retired players per year
player_prize_money_adjust.loc[player_prize_money_adjust["status"] == "retired"].groupby("year")["player_id"].nunique()

,player_id
year,
1956,1
1985,1
1987,1
1988,1
1989,1
1990,2
1991,3
1992,4
1993,5


In [15]:
# Aggregate career statistics for retired players
retired_players = (
    player_prize_money_adjust
    .loc[player_prize_money_adjust["status"] == "retired"]
    .groupby("player_id", as_index=False)
    .agg(
        years_on_tour=("year", "nunique"),
        total_prize_usd_2025=("prize_usd_2025", "sum"),
        first_year=("year", "min"),
        last_year=("year", "max")
    )
)

In [16]:
# Display descriptive statistics for retired players
retired_players.describe()

,years_on_tour,total_prize_usd_2025,first_year,last_year
count,3681.000000,3.681000e+03,3681.000000,3681.000000
mean,7.236892,4.782739e+05,2011.885086,2019.034230
std,4.900461,4.605599e+06,5.730932,3.203246
min,1.000000,2.300000e+02,1956.000000,2014.000000
25%,3.000000,2.028000e+03,2008.000000,2016.000000
50%,6.000000,7.035000e+03,2012.000000,2019.000000
75%,10.000000,3.534300e+04,2015.000000,2022.000000
max,27.000000,1.700654e+08,2024.000000,2024.000000


In [17]:
# Display top 5 retired players by years on tour
retired_players.sort_values("years_on_tour", ascending=False).head()

,player_id,years_on_tour,total_prize_usd_2025,first_year,last_year
2953,s550,27,1072730.0,1992,2018
1844,l397,27,21918499.0,1997,2023
2533,p605,25,649024.0,2000,2024
1007,f324,25,170065353.0,1997,2021
2664,q019,25,517294.0,1998,2024


In [18]:
# Display top 5 retired players by total adjusted prize money
retired_players.sort_values("total_prize_usd_2025", ascending=False).head()

,player_id,years_on_tour,total_prize_usd_2025,first_year,last_year
1007,f324,25,170065353.0,1997,2021
2334,n409,24,160795558.0,2001,2024
2069,mc10,23,78658477.0,2002,2024
1012,f401,21,40940936.0,1999,2019
265,ba47,19,37465597.0,2001,2019


In [19]:
#Load ATP players overview
players_overview = pd.read_csv('https://raw.githubusercontent.com/nicolasrojasv/nicolasrojasv.github.io/refs/heads/main/Project/data/players_characteristics.csv')

In [20]:
#Complete name of the player
players_overview['name'] = players_overview['FirstName'] + ' ' + players_overview['LastName']

In [22]:
#Merge players prize by year with the player name an NatlId y Nationality
retired_players = pd.merge(retired_players, players_overview[['player_id', 'NatlId', 'Nationality', 'name']], on = 'player_id', how='left')

In [32]:
#Save retired players data
retired_players.to_csv('retired_players.csv', index=False)

In [26]:
# Create and display a scatter plot of career duration vs. cumulative prize money for retired players
chart = (
    alt.Chart(retired_players)
    .mark_circle(size=70, opacity=0.7, color="#bdbdbd")
    .encode(
        x=alt.X(
            "total_prize_usd_2025:Q",
            title="Cumulative prize money (USD, 2025)",
            axis=alt.Axis(format=",.0f"),
            scale=alt.Scale(zero=True)
        ),
        y=alt.Y(
            "years_on_tour:Q",
            title="Career duration (years)",
            scale=alt.Scale(zero=False)
        ),
        tooltip=[
            alt.Tooltip("name:N", title="Player: "),
            alt.Tooltip("years_on_tour:Q", title="Years on tour"),
            alt.Tooltip(
                "total_prize_usd_2025:Q",
                title="Cumulative prize (USD)",
                format=",.0f"
            ),
            alt.Tooltip("first_year:Q", title="First year"),
            alt.Tooltip("last_year:Q", title="Last year")
        ]
    )
    .properties(
        width=480,
        height=320,
        title="Career duration and cumulative prize money of retired players"
    )
)

chart

alt.Chart(...)

In [27]:
# Calculate annual and total career costs for retired players
ANNUAL_COST_2025 = 113_494
retired_players = retired_players.assign(
    annual_cost_2025=ANNUAL_COST_2025,
    total_cost_2025=lambda d: d["years_on_tour"] * d["annual_cost_2025"]
)

In [28]:
# Calculate the career balance (prize money minus cost) for retired players
retired_players["career_balance_2025"] = (
    retired_players["total_prize_usd_2025"]
    - retired_players["total_cost_2025"]
)

In [29]:
# Display the first 10 rows of retired players with calculated costs and balance
retired_players.head(10)

,player_id,years_on_tour,total_prize_usd_2025,first_year,last_year,NatlId,Nationality,name,annual_cost_2025,total_cost_2025,career_balance_2025
0,a09r,9,7647.0,2015,2023,TUR,Turkiye,Umut Akkoyun,113494,1021446,-1013799.0
1,a0a1,3,1387.0,2015,2022,EGY,Egypt,Mohamed Aboulkassem,113494,340482,-339095.0
2,a0a6,4,3457.0,2017,2023,POR,Portugal,Tomas Almeida,113494,453976,-450519.0
3,a0an,10,30600.0,2015,2024,TUR,Turkiye,Cengiz Aksu,113494,1134940,-1104340.0
4,a0ao,7,80180.0,2015,2021,RUS,Russia,Alen Avidzba,113494,794458,-714278.0
5,a0au,3,767.0,2016,2019,SWE,Sweden,Jakob Arvidsson Ehn,113494,340482,-339715.0
6,a0ay,3,3296.0,2016,2019,EGY,Egypt,Mohamed Abdel-Aziz,113494,340482,-337186.0
7,a0b6,9,73752.0,2016,2024,FRA,France,Jaimee Floyd Angele,113494,1021446,-947694.0
8,a0bd,6,6626.0,2016,2022,ITA,Italy,Gianluca Acquaroli,113494,680964,-674338.0
9,a0bg,6,12175.0,2016,2021,AUT,Austria,Jakob Aichhorn,113494,680964,-668789.0


In [30]:
# Display the 5 retired players with the highest career balance
retired_players.sort_values("career_balance_2025").tail()

,player_id,years_on_tour,total_prize_usd_2025,first_year,last_year,NatlId,Nationality,name,annual_cost_2025,total_cost_2025,career_balance_2025
265,ba47,19,37465597.0,2001,2019,CZE,Czechia,Tomas Berdych,113494,2156386,35309211.0
1012,f401,21,40940936.0,1999,2019,ESP,Spain,David Ferrer,113494,2383374,38557562.0
2069,mc10,23,78658477.0,2002,2024,GBR,Great Britain,Andy Murray,113494,2610362,76048115.0
2334,n409,24,160795558.0,2001,2024,ESP,Spain,Rafael Nadal,113494,2723856,158071702.0
1007,f324,25,170065353.0,1997,2021,SUI,Switzerland,Roger Federer,113494,2837350,167228003.0


In [31]:
# Filter retired players with total prize money up to $10,000,000 for modeling
low_earning_retired_players = (
    retired_players
    .loc[retired_players["total_prize_usd_2025"] <= 10_000_000]
    .copy()
)

In [33]:
# Define features (X) and target variable (y) for the model
X = low_earning_retired_players[["total_prize_usd_2025"]]
y = low_earning_retired_players["years_on_tour"]

In [34]:
# Import StandardScaler for feature scaling
from sklearn.preprocessing import StandardScaler

In [35]:
# Scale the features (X) and target variable (y) using StandardScaler
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1)).ravel()

In [36]:
# Import train_test_split for splitting data into training and testing sets
from sklearn.model_selection import train_test_split

In [37]:
# Split the scaled data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_scaled,
    test_size=0.3,
    random_state=42
)

In [38]:
# Import RandomForestRegressor for building the regression model
from sklearn.ensemble import RandomForestRegressor

In [39]:
# Initialize and train a RandomForestRegressor model
model = RandomForestRegressor(
    n_estimators=300,
    random_state=42
)

model.fit(X_train, y_train)

RandomForestRegressor(n_estimators=300, random_state=42)

In [40]:
# Import R-squared and Mean Squared Error metrics for model evaluation
from sklearn.metrics import r2_score, mean_squared_error

In [41]:
# Make predictions with RandomForest model and evaluate its performance
y_pred = model.predict(X_test)

r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

r2, rmse

(0.7283363477788516, np.float64(0.5420405257124928))

In [42]:
# Inverse transform predicted and actual values to their original scale
y_test_original = scaler_y.inverse_transform(y_test.reshape(-1, 1)).ravel()
y_pred_original = scaler_y.inverse_transform(y_pred.reshape(-1, 1)).ravel()

In [43]:
# Import LinearRegression for comparison
from sklearn.linear_model import LinearRegression

In [44]:
# Initialize, train, and evaluate a Linear Regression model
lin_model = LinearRegression()
lin_model.fit(X_train, y_train)

y_pred_lin = lin_model.predict(X_test)

r2_lin = r2_score(y_test, y_pred_lin)
rmse_lin = np.sqrt(mean_squared_error(y_test, y_pred_lin))

r2_lin, rmse_lin

(0.20234168280626263, np.float64(0.9288047746562738))

In [45]:
# Visualize actual vs. predicted career duration with RandomForest model metrics
pred_df = pd.DataFrame({
    "years_actual": y_test_original,
    "years_predicted": y_pred_original
})

scatter = (
    alt.Chart(pred_df)
    .mark_circle(size=70, opacity=0.6, color="#0033A0")
    .encode(
        x=alt.X(
            "years_predicted:Q",
            title="Predicted years on tour"
        ),
        y=alt.Y(
            "years_actual:Q",
            title="Actual years on tour"
        ),
        tooltip=[
            alt.Tooltip("years_actual:Q", title="Actual years"),
            alt.Tooltip("years_predicted:Q", title="Predicted years")
        ]
    )
)

reference_line = (
    alt.Chart(pred_df)
    .mark_line(strokeDash=[4, 4], color="#7f7f7f")
    .encode(
        x="years_actual:Q",
        y="years_actual:Q"
    )
)

metrics_text = (
    alt.Chart(
        pd.DataFrame({
            "text": [f"R² = {r2*100:.1f}%\nRMSE = {rmse:.2f} years"]
        })
    )
    .mark_text(
        align="left",
        baseline="top",
        fontSize=13,
        color="#303030"
    )
    .encode(
        x=alt.value(10),
        y=alt.value(10),
        text="text:N"
    )
)

chart = (
    scatter + reference_line + metrics_text
).properties(
    width=420,
    height=320,
    title="Actual vs. predicted career duration"
)

chart

alt.LayerChart(...)

In [46]:
#Save the actual and predicted data
pred_df.to_csv("actual_vs_predicted_data.csv", index=False)

In [47]:
# Build metrics DataFrame
metrics_df = pd.DataFrame({
    "metric": ["R2_percentage", "RMSE_years"],
    "value": [r2 * 100, rmse]
})

# Optional: round for presentation
metrics_df["value"] = metrics_df["value"].round(2)

# Save to CSV
metrics_df.to_csv("model_performance_metrics.csv", index=False)